In [ ]:
!pip install mwclient

: 

In [ ]:
pip install tensorflow

: 

In [ ]:
import mwclient
import time

site=mwclient.Site("en.wikipedia.org")
page=site.pages["Bitcoin"]

: 

In [ ]:
revs=list(page.revisions())

: 

In [ ]:
revs[0]

: 

In [ ]:
from transformers import pipeline
sentiment_pipeline=pipeline("sentiment-analysis")

def find_sentiment(text):
    sent=sentiment_pipeline([text[:250]])[0]
    score=sent["score"]
    if sent["label"]=="NEGATIVE":
        score *= -1
    return score

: 

In [ ]:
from statistics import mean

edits = {}
for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
        
    edits[date]["edit_count"] += 1
    
    # Check if 'comment' key exists in the revision
    if "comment" in rev:
        comment = rev["comment"]
        edits[date]["sentiments"].append(find_sentiment(comment))
    else:
        # Handle the case where 'comment' does not exist
        pass  # or some other logic

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_+sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
        
    del edits[key]["sentiments"]

# Print the final edits dictionary
print(edits)


: 

In [ ]:
import pandas as pd
edits_df=pd.DataFrame.from_dict(edits, orient="index")

: 

In [ ]:
edits_df

: 

In [ ]:
edits_df.index=pd.to_datetime(edits_df.index)

: 

In [ ]:
from datetime import datetime
dates=pd.date_range(start="2009-03-08", end=datetime.today())

: 

In [ ]:
dates

: 

In [ ]:
edits_df=edits_df.reindex(dates, fill_value=0)

: 

In [ ]:
edits_df

: 

In [ ]:
rolling_edits=edits_df.rolling(30).mean()

: 

In [ ]:
rolling_edits

: 

In [ ]:
rolling_edits=rolling_edits.dropna()

: 

In [ ]:
rolling_edits

: 

In [ ]:
rolling_edits.to_csv("wikipedia_edits.csv")

: 

: 

: 